# Deleting Data from SQLite Tables

In this notebook, you'll learn how to delete data from SQLite tables using DELETE statements. We'll cover basic deletes, conditional deletes, and best practices for safe data removal.

In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect('university.db')
cursor = conn.cursor()

print('Connected to database')

## Basic DELETE Syntax

The basic syntax for DELETE is:

```sql
DELETE FROM table_name WHERE condition;
```

**WARNING**: Omitting WHERE clause deletes ALL rows!

In [ ]:
# First, let's see current student count
cursor.execute('SELECT COUNT(*) FROM students')
count_before = cursor.fetchone()[0]
print(f'Students before delete: {count_before}')

# Delete a specific student
cursor.execute('DELETE FROM students WHERE id = 3')

print('Deleted student with ID 3')
print(f'Rows affected: {cursor.rowcount}')

## Conditional Deletes

Use WHERE clause with various conditions to delete specific rows.

In [ ]:
# Delete students with low grades
cursor.execute('DELETE FROM students WHERE grade < 3.0')

print('Deleted students with grade < 3.0')
print(f'Rows affected: {cursor.rowcount}')

# Delete old enrollments
cursor.execute('DELETE FROM enrollments WHERE semester = "Fall 2022"')

print('Deleted old enrollments')
print(f'Rows affected: {cursor.rowcount}')

## Using Parameters in Deletes

Always use parameters for user input to prevent SQL injection.

In [ ]:
# Delete using parameters
student_name = 'Charlie Brown'
cursor.execute('DELETE FROM students WHERE name = ?', (student_name,))

print(f'Deleted student: {student_name}')
print(f'Rows affected: {cursor.rowcount}')

# Delete using named parameters
cursor.execute('DELETE FROM courses WHERE department = :dept', {'dept': 'PHYS'})

print('Deleted Physics courses')
print(f'Rows affected: {cursor.rowcount}')

## DELETE with Subqueries

Use subqueries to delete based on related data.

In [ ]:
# Delete enrollments for students not in students table (orphaned records)
cursor.execute('''
    DELETE FROM enrollments
    WHERE student_id NOT IN (SELECT id FROM students)
''')

print('Deleted orphaned enrollments')
print(f'Rows affected: {cursor.rowcount}')

# Delete courses with no enrollments
cursor.execute('''
    DELETE FROM courses
    WHERE course_id NOT IN (SELECT DISTINCT course_id FROM enrollments)
''')

print('Deleted courses with no enrollments')
print(f'Rows affected: {cursor.rowcount}')

## DELETE with LIMIT

Use LIMIT to delete a specific number of rows.

In [ ]:
# Insert some test data first
test_students = [
    ('Test Student 1', 25, 2.5),
    ('Test Student 2', 26, 2.7),
    ('Test Student 3', 27, 2.8)
]

cursor.executemany('INSERT INTO students (name, age, grade) VALUES (?, ?, ?)', test_students)
print('Inserted test students')

# Delete only 2 students with lowest grades
cursor.execute('''
    DELETE FROM students
    WHERE id IN (
        SELECT id FROM students
        WHERE grade < 3.0
        ORDER BY grade ASC
        LIMIT 2
    )
''')

print('Deleted 2 students with lowest grades')
print(f'Rows affected: {cursor.rowcount}')

## Checking Delete Results

Verify that deletes were applied correctly.

In [ ]:
# Check remaining students
cursor.execute('SELECT COUNT(*) FROM students')
count_after = cursor.fetchone()[0]
print(f'Students after deletes: {count_after}')

# List remaining students
cursor.execute('SELECT id, name, grade FROM students ORDER BY id')
remaining_students = cursor.fetchall()

print('\nRemaining students:')
for student in remaining_students:
    print(f'ID: {student[0]}, Name: {student[1]}, Grade: {student[2]}')

# Check other tables
cursor.execute('SELECT COUNT(*) FROM courses')
courses_count = cursor.fetchone()[0]
print(f'\nRemaining courses: {courses_count}')

cursor.execute('SELECT COUNT(*) FROM enrollments')
enrollments_count = cursor.fetchone()[0]
print(f'Remaining enrollments: {enrollments_count}')

## Safe Delete Practices

- Always backup data before deleting
- Use transactions for multiple deletes
- Test deletes on development data first
- Use SELECT before DELETE to verify what will be deleted
- Consider soft deletes (adding a 'deleted' flag) for important data

In [ ]:
# Example of safe delete with preview
def safe_delete(cursor, table, condition):
    # First, show what will be deleted
    preview_query = f'SELECT * FROM {table} WHERE {condition}'
    cursor.execute(preview_query)
    to_delete = cursor.fetchall()
    
    if to_delete:
        print(f'Will delete {len(to_delete)} rows from {table}:')
        for row in to_delete[:3]:  # Show first 3
            print(f'  {row}')
        if len(to_delete) > 3:
            print(f'  ... and {len(to_delete) - 3} more')
        
        # Ask for confirmation (simulated)
        confirm = input('Proceed with delete? (y/n): ').lower().strip()
        if confirm == 'y':
            delete_query = f'DELETE FROM {table} WHERE {condition}'
            cursor.execute(delete_query)
            print(f'Deleted {cursor.rowcount} rows')
            return cursor.rowcount
        else:
            print('Delete cancelled')
            return 0
    else:
        print('No rows match the delete condition')
        return 0

# Example usage (commented out to avoid input prompt)
# safe_delete(cursor, 'students', 'grade < 2.0')

print('Safe delete function defined')

## TRUNCATE vs DELETE

SQLite doesn't have TRUNCATE TABLE command. Use DELETE FROM table_name to remove all rows.

In [ ]:
# Create a test table to demonstrate
cursor.execute('''
    CREATE TABLE test_data (
        id INTEGER PRIMARY KEY,
        value TEXT
    )
''')

# Insert test data
test_data = [('A',), ('B',), ('C',)]
cursor.executemany('INSERT INTO test_data (value) VALUES (?)', test_data)

print('Created test table with data')

# Delete all rows (equivalent to TRUNCATE)
cursor.execute('DELETE FROM test_data')

print('Deleted all rows from test table')
print(f'Rows affected: {cursor.rowcount}')

# Drop the test table
cursor.execute('DROP TABLE test_data')
print('Dropped test table')

## Summary

In this notebook, you learned how to:

- Perform basic DELETE operations
- Use WHERE clauses for conditional deletes
- Use parameterized queries for safe deletes
- Delete with subqueries
- Use LIMIT with deletes
- Verify delete results
- Implement safe delete practices
- Remove all rows from tables

Remember: DELETE operations are permanent (unless within a transaction that can be rolled back). Always backup important data before deleting, and test your delete conditions carefully.

In [ ]:
# Commit all changes
conn.commit()

# Close the connection
conn.close()
print('Database connection closed')